Data Wrangling Notebook for VertNet Mammal Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import re
import uuid 

Import Vertnet Mammal Data

In [2]:
mammal = pd.read_csv("./../Original Data/no_bats_2020-08-12b.csv")

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32,33,35,37,38,40,41,42,44,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,77,78,79,80,81,82,83,84,85,86,87,88,89,90,92,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,132,133,134,135,136,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,160,161,164,165,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,230,231,232,233,235,239,241,243,244,245,248,249,251,254,255,256,260,261,262,263,265,267,268,269,270,271,272,274,275,276,277,279,280,281,284,287,291,292,294,295,296,297,29

Clean yearCollected column

In [3]:
# Filling N/As with "Unknown"
mammal["eventdate"] = mammal["eventdate"].fillna("Unknown")

# Create yearCollected Column
mammal = mammal.assign(yearCollected = '')

# Creating event date variable
verbatim_date = mammal['eventdate']

# Establishing vertnet filter
vertnet_date_filter = verbatim_date.str.contains("""IV|0000|September|<|NW|latter|unknown|(MCZ)|(MSU)|present|
                                                    and|;|&|mainly|between|Between|BETWEEN|OR|Unknown|UNKNOWN|
                                                    #|TO|\?|\'|----|19--|No Date|\,|\d{4}-\d{4}|(/n) /d|\d{4}[s]|
                                                    \d{4}\'[S]|1075-07-29|975-07-17|2088|9999|0201|1197|
                                                    1260|4560|1024|1119|1192|1072|1186""")

# Grabbing clean data
verbatim_date_clean= verbatim_date[vertnet_date_filter==False]


# Captures year within string
def year_search(year):
    """Search string for 4 digit number and pass to correct function"""
    if (re.search(r'\d{4}$', year)):
        return year_cleaner_front(year)
    elif (re.search(r'^\d{4}', year)):
        return year_cleaner_back(year)

def year_cleaner_front(year):
    """Isolate the year at the beginning of the string"""
    cleaned_year = year[len(year)-4:len(year)]
    return cleaned_year

def year_cleaner_back(year):
    """Isolate the year at the end of the string"""
    cleaned_year = year[0:4]
    return cleaned_year

mammal["yearCollected"] = verbatim_date_clean.apply(year_search)
mammal["yearCollected"] = mammal["yearCollected"].fillna("Unknown")


/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Clean up lifeStage

In [4]:
# Fill in NA
mammal["lifestage_cor"] = mammal['lifestage_cor'].fillna("Not Collected")

# Create Filters
adult = mammal['lifestage_cor']=="Adult"
juvenile = mammal['lifestage_cor']=="Juvenile"
ns = mammal['lifestage_cor']=="NS"

# Assign correct terms using filters
mammal['lifestage_cor'][adult] = "adult"
mammal['lifestage_cor'][juvenile] = "juvenile"
mammal['lifestage_cor'][ns] = "Not Collected"

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

Clean up sex column

In [5]:
# Clean up sex column 
female = mammal['sex']=="female"
male = mammal['sex']=="male"
mammal['sex'][(female == False)&(male==False)]=""

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### TO DO
Figure out what is going on with scientific name. You've got nothing here

In [6]:
#mammal["binomial"]=mammal["binomial"].fillna("Unknown")

Adding additional required GEOME columns

In [7]:
mammal=mammal.assign(samplingProtocol="Unknown")
mammal=mammal.assign(measurementMethod="Unknown")
mammal=mammal.assign(basisOfRecord="PreservedSpecimen")

Create verbatimEventDate column

In [8]:
mammal=mammal.assign(verbatimEventDate = '')
mammal['verbatimEventDate']=mammal['eventdate']

Clean up country column 

In [31]:
mammal['ind'] = np.arange(len(mammal))

In [39]:
#Append countries to verbatim locality column
mammal["locality"] = mammal["locality"] + "," + mammal["country"]

#Read GEOME country list
geome_countries = pd.read_csv("./../Mapping Files/geome_country_list.csv")

country_dictionary = {"United States":"USA", "U S A":"USA", 
                      "Philippine Islands":"Philippines",
                      "Indonesia; Borneo":"Indonesia",
                      "Malaysia; Malaya":"Malaysia",
                      "U.S. Virgin Islands":"Virgin Islands",
                      "Republic of South Africa":"South Africa",
                      "Ivory Coast":"Cote d'Ivoire",
                      "Federated States of Micronesia":"Micronesia",
                      "Lesser Antilles; Grenada":"Grenada",
                      "Indonesia; Java":"Indonesia",
                      "Lesser Antilles; Saint Vincent":"Saint Vincent and the Grenadines",
                      "Lesser Antilles; Barbados":"Barbados",
                      "ST VINCENT":"Saint Vincent and the Grenadines",
                      "Lesser Antilles; Montserrat": "Montserrat",
                      "Indonesia; Sumatra":"Indonesia",
                      "Virgin Islands, US":"Virgin Islands",
                      "Lesser Antilles; Antigua":"Antigua and Barbuda",
                      "England":"United Kingdom",
                      "Republic of Trinidad and Tobago":"Trinidad and Tobago",
                      "Trinidad And Tobago; Trinidad":"Trinidad and Tobago",
                      "COMMONWEALTH OF THE NORTHERN MARIANA ISLANDS":"Northern Mariana Islands",
                      "Congo":"Democratic Republic of the Congo",
                      "Malaysia; Sabah":"Malaysia",
                      "Lesser Antilles; Martinique":"Martinique",
                      "Republic of the Marshall Islands":"Marshall Islands",
                      "Commonwealth of the Bahamas":"Bahamas",
                      "Trinidad & Tabago":"Trinidad and Tobago",
                      "United Kingdom; England":"United Kingdom",
                      "United Kingdom; Scotland":"United Kingdom",
                      "United Kingdom; Wales":"United Kingdom",
                      "Lesser Antilles; Dominica":"Dominica",
                      "Papua, New Guinea":"Papua New Guinea",
                      "People's Republic of China":"China",
                      "SCOTLAND":"United Kingdom"}

def country_correction(country, ind): 
    """Corrects country column to geome specific country list"""
    if country in geome_countries.values:
        return country
    elif country in country_dictionary.keys():
        country = country_dictionary[country]
        return country 
    else:
        country = "Unknown"
        return country 
    
mammal['country'] = mammal.apply(lambda x: country_correction(x.country, x.ind), axis=1)
#mammal["country"] = mammal[["country","index"]].apply(country_correction)

# Fill N/A with unknown
#mammal["country"]=mammal["country"].fillna("Unknown")

KeyboardInterrupt: 

Create verbatimElevation Column

In [95]:
string_max = mammal["maximumelevationinmeters"].astype(str)
string_min = mammal["minimumelevationinmeters"].astype(str)
mammal['verbatimElevation']= string_max + "," + string_min

Rearrange columns so that template columns are first, followed by measurement values

In [96]:
# Create column list
cols = mammal.columns.tolist()

# Specify desired columns
cols = ['catalognumber',
        'collectioncode',
        'decimallatitude',
        'decimallongitude',
        'verbatimElevation',
        'institutioncode',
        'verbatimEventDate',
        'locality',
        'samplingProtocol',
        'measurementMethod',
        'country',
        'sex',
        'lifestage_cor',
        'binomial',
        'basisOfRecord',
        'yearCollected',
        'body_mass.1.value',
        'ear_length.1.value',
        'hind_foot_length.1.value',
        'tail_length.1.value',
        'total_length.1.value']

# Subset dataframe
mammal = mammal[cols]

Matching template and column terms

In [97]:
# Renaming columns 
mammal = mammal.rename(columns = {'catalognumber': 'catalogNumber',
                                 'collectioncode':'collectionCode',
                                 'decimallatitude':'decimalLatitude',
                                 'decimallongitude':'decimalLongitude',
                                 'maximumelevationinmeters':'maximumElevationInMeters',
                                 'minimumelevationinmeters':'minimumElevationInMeters',
                                 'institutioncode' :'institutionCode',
                                 'locality':'verbatimLocality',
                                 'lifestage_cor':'lifeStage',
                                 'binomial':'scientificName'})

Matching trait and ontology terms

In [98]:
# Renaming columns
mammal = mammal.rename(columns={'body_mass.1.value':'body mass',
                                'ear_length.1.value': 'ear length to notch',
                                'hind_foot_length.1.value':'pes length',
                                'tail_length.1.value':'tail length',
                                'total_length.1.value':'body length'})

Create materialSampleID which is a UUID for each measurement

In [99]:
mammal=mammal.assign(materialSampleID = '')
mammal['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(mammal.index))]

Create eventID and populate it with materialSampleID

In [100]:
mammal=mammal.assign(eventID = mammal["materialSampleID"])

Add required GEOME column locality after reassigning locality to verbatimLocality

In [101]:
mammal=mammal.assign(locality="Unknown")

Creating long version, first specifiying keep variables, then naming type and value

In [187]:
longVersMammal=pd.melt(mammal,
                      id_vars=['catalogNumber',
                      'collectionCode',
                      'decimalLatitude',
                      'decimalLongitude',
                      'verbatimElevation',
                      'yearCollected',
                      'basisOfRecord',
                      'verbatimEventDate',
                      'institutionCode',
                      'lifeStage',
                      'verbatimLocality',
                      'locality',
                      'samplingProtocol',
                      'measurementMethod',
                      'country',
                      'sex',
                      'scientificName',
                      'materialSampleID',
                      'eventID'], 
                var_name = 'measurementType',
                value_name = 'measurementValue')

Populating measurementUnit column with appropriate measurement units in long version

In [188]:
# Create measurementUnit column
longVersMammal=longVersMammal.assign(measurementUnit="")

#Create filters
long_body_mass_filter=longVersMammal['measurementType']=="body mass"
long_no_body_filter=longVersMammal['measurementType']!="body mass"

#Assign units using filters
longVersMammal['measurementUnit'][long_body_mass_filter] = "g"
longVersMammal['measurementUnit'][long_no_body_filter] = "mm"


Create diagnosticID which is a unique number for each measurement

In [189]:
longVersMammal=longVersMammal.assign(diagnosticID = '')
longVersMammal['diagnosticID'] = np.arange(len(longVersMammal))

If measurement value equals N/A, delete entire row. Drop range values. 

In [193]:
#Drop N/A
longVersMammal = longVersMammal.dropna(subset=['measurementValue'])

#Drop Range Values
range_value_filter=longVersMammal['measurementValue'].str.contains(",|one", na=False)
longVersMammal['measurementValue'][range_value_filter] = float("nan")
longVersMammal = longVersMammal.dropna(subset=['measurementValue'])


Breaking up the data into more managable sizes for validation and DE storage

In [205]:
# Create chunks list
chunks = []

# Separating files into chunks of ~500,000
chunks = np.array_split(longVersMammal, 6)


Creating data chunks

In [ ]:
for i in range(len(chunks)):
    new=i+1
    chunks[i].to_csv('../Mapped Data/FuTRES_Mammals_VertNet_Global_Modern_'+ str(new) +'.csv', index=False)
    print("mapped_data",i, " done")